In [1]:
import os
os.chdir('C:\\Users\\CrankMuffler\\Documents\\GitHub\\RA2_alpine_lads')

In [2]:
from utils.config import Config

test_config = Config('./utils/stadlerm_config.json')

In [3]:
from dataset.train_dataset import train_dataset

dataset = train_dataset(test_config)

hands_dataset, feet_dataset, hands_dataset_val, feet_dataset_val = dataset.initialize_pipeline()

In [4]:
feet_dataset

<PrefetchDataset shapes: ((None, 378, 512, 1), (None, 13)), types: (tf.float32, tf.int64)>

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [6]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(test_config.img_height,test_config.img_width ,1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(13)
])

In [7]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 378, 512, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 189, 256, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 189, 256, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 94, 128, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 94, 128, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 47, 64, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 192512)            0

In [9]:
history = model.fit(
    feet_dataset,validation_data=feet_dataset_val,epochs=10, steps_per_epoch=50,
    validation_steps=2
)

Train for 50 steps, validate for 2 steps
Epoch 1/10
50/50 [==============================] - 166s 3s/step - loss: 5.2719 - val_loss: 13.4768
Epoch 2/10
50/50 [==============================] - 122s 2s/step - loss: 4.4555 - val_loss: 3.5005
Epoch 3/10
50/50 [==============================] - 122s 2s/step - loss: 3.8054 - val_loss: 11.1349
Epoch 4/10
50/50 [==============================] - 138s 3s/step - loss: 3.8231 - val_loss: 11.9856
Epoch 5/10
50/50 [==============================] - 139s 3s/step - loss: 4.6218 - val_loss: 7.2555
Epoch 6/10
50/50 [==============================] - 138s 3s/step - loss: 4.1897 - val_loss: 10.3944
Epoch 7/10
50/50 [==============================] - 141s 3s/step - loss: 3.8359 - val_loss: 6.7489
Epoch 8/10
50/50 [==============================] - 140s 3s/step - loss: 4.1818 - val_loss: 11.3190
Epoch 9/10
50/50 [==============================] - 163s 3s/step - loss: 3.5443 - val_loss: 10.6808
Epoch 10/10
50/50 [==============================] - 133s 3s/s

In [19]:
for image, label in feet_dataset_val.take(5):
    N = image.shape[0]
    D = label.shape[1]
    
    error = np.zeros((N, D))
    
    for n in range(N):
        y_pred = model.predict(np.array(image[n]).reshape(1,test_config.img_height,test_config.img_width,1))
        y = label[n]
        
        
        E[n, :] = np.square(y_pred - y)
        
    rmse = np.sqrt(np.mean(E, axis = 0))
        
    print("Batch RMSE: ", rmse)

Batch RMSE:  [ 0.90138782  1.08972474  1.03077641  1.11803399  1.          1.03077641
  1.03077641  0.70710678  1.11803399  1.22474487  1.11803399  1.11803399
 10.43132302]
Batch RMSE:  [0.         0.         0.75       0.90138782 0.5        1.11803399
 1.         0.90138782 0.90138782 0.70710678 0.90138782 0.70710678
 6.03634823]
Batch RMSE:  [ 0.8291562   0.79056942  1.08972474  1.19895788  1.03077641  1.25
  1.3462912   1.          1.22474487  1.22474487  1.22474487  1.32287566
 11.14955156]
Batch RMSE:  [ 0.8291562   0.75        1.27475488  1.52069063  1.22474487  1.88745861
  1.14564392  1.36930639  1.62018517  1.41421356  1.62018517  1.41421356
 12.97593927]
Batch RMSE:  [ 0.79056942  0.79056942  1.11803399  1.11803399  1.11803399  1.41421356
  1.03077641  1.03077641  1.25        1.11803399  1.25        1.22474487
 11.21383075]
